In [ ]:
import sys
sys.path.append('../util')

from util import get_max_db_id
from util import get_time_span

In [ ]:
STT_NS_GA = 5
STT_NS_GB = 8
STT_NS_GG = 14
STT_NS_GH = 15

group_id = STT_NS_GB
target_directory = f"segments_ns"
last_db_id = get_max_db_id()

In [ ]:
from transformers import pipeline
from tqdm.auto import tqdm

from pathlib import Path

target_path = Path(target_directory)

generator = pipeline(model="spsither/wav2vec2_run9.15")
rows = []

for file in tqdm(target_path.glob('*.wav'), total=len(list(target_path.glob('*.wav')))):
    inf = generator(str(file))["text"]
    rows.append([file.stem, f"https://d38pmlk0v88drf.cloudfront.net/wav16k/{file.name}", inf , get_time_span(str(file.name))])

In [ ]:
import pandas as pd
df = pd.DataFrame(rows, columns =['file_name', 'url', 'inference_transcript', 'audio_duration'])

In [ ]:
df = df.sort_values('file_name').reset_index(drop=True)

In [ ]:
df[['inference_transcript','url']].iloc[0:10].to_dict()

In [ ]:
df['group_id'] = group_id
df['state'] = 'transcribing'
df['id'] = df.index + last_db_id + 1

In [ ]:
df.head()

In [ ]:
df['file_name'] = df['file_name'].str.replace(".wav", "")

In [ ]:
df.to_csv(f"segments_ns_{group_id}.csv", index=False)

In [ ]:
df[df['inference_transcript'].apply(lambda x: len(x) < 500)].shape

In [ ]:
# ! rm -rf full_audio after_split segments_ns segments_ns_*.csv